In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

## Extract Flight.csv into DataFrame and CleanUp

In [2]:
flight = pd.read_csv("Resource/flight_data.csv")

# Only keep JFK airport info
flight_JFK = flight.loc[flight["origin"] == "JFK"]  

# Filter columns "dep_delay" and "arr_delay", only keep delayed flights (>0)
flight_JFK_delayed = flight_JFK.loc[flight_JFK["dep_delay"] >0.]  
flight_JFK_delayed = flight_JFK_delayed.loc[flight_JFK_delayed["arr_delay"] >0.]  

# Create a filtered dataframe from specific columns
flight_JFK_delayed_df = pd.DataFrame(data = {"Month": flight_JFK_delayed["month"],
                                          "Day": flight_JFK_delayed["day"],
                                          "Departure Delay": flight_JFK_delayed["dep_delay"],
                                          "Arrival Delay": flight_JFK_delayed["arr_delay"],                               
                                          "Tailnum": flight_JFK_delayed["tailnum"]})

# flight_JFK_delayed_df.head()

## Transform dataframe to get daily total number of delayed flights 

In [3]:
Time = []
flight_JFK_delay_number = []

Month = 12 
for m in range(1, Month+1):
    days = pd.Period("2013-{:02d}".format(m)).days_in_month #days in a month
    for d in range(1, days+1):
        Time.append("2013-%02d-%02d"% (m,d))
        number = flight_JFK_delayed_df[(flight_JFK_delayed_df["Month"] == m) \
                                       & (flight_JFK_delayed_df["Day"] ==d )].shape[0]
        flight_JFK_delay_number.append(number)
        
# Create a new DataFrame 
flight_JFK_delaynumber_df = pd.DataFrame(data = {"Time" : Time,
                            "delayed flights #" : flight_JFK_delay_number})

## Extract 2013 weather variables into DataFrame and CleanUp

#### Part I: Loop through weather variables and average each by days 

In [4]:
weather_variables = ["temperature", "humidity", "pressure", "wind_speed", "wind_direction"]

for var in weather_variables:
    print ("processing "+var)
    var_df_ori = pd.read_csv("Resource/"+var+".csv")

    # Only extract data for NYC
    var_df_nyc = pd.DataFrame(data ={"datetime": var_df_ori ["datetime"],
                                      var: var_df_ori ["New York"]})
    # Only extract data in 2013
    var_df_nyc_2013 = var_df_nyc.loc[ pd.to_datetime(var_df_nyc["datetime"]).dt.year ==2013]  

    # Get daily averaged value 
    # (This code line is so interesting!)
    var_df_nyc_2013_daily = var_df_nyc_2013.groupby(np.arange(len(var_df_nyc_2013))//24).mean()

    # Append humidity to flight_JFK_delaynumber_df
    flight_JFK_delaynumber_df[var] = var_df_nyc_2013_daily.round(1)

processing temperature
processing humidity
processing pressure
processing wind_speed
processing wind_direction


#### Part II: Weather descrption strings sort by daily frequency (Think thoroughly!)

In [5]:
# Now let's deal with weather_description, since they are strings and can't be averaged by day, 
# we select the most frequent weather_description by day
wd_df_ori = pd.read_csv("Resource/weather_description.csv")
wd_df_nyc = pd.DataFrame(data ={"datetime": wd_df_ori ["datetime"],
                                "weather_description": wd_df_ori ["New York"]})

wd_df_nyc_2013 = wd_df_nyc.loc[ pd.to_datetime(wd_df_nyc["datetime"]).dt.year ==2013]  

wd_df_nyc_2013_daily = []
for d in range(365):
    wd_nyc_daily = wd_df_nyc_2013.iloc[ d*24:(d+1)*24, 1]  
    wd_daily = str(wd_nyc_daily.mode()).split()
    wd_df_nyc_2013_daily.append(' '.join(wd_daily[1:-2]))

# Append to flight_JFK_delaynumber_df
flight_JFK_delaynumber_df["Weather Description"] = pd.Series(wd_df_nyc_2013_daily) 

## Load DataFrames into database

In [6]:
# Rename column names and add units 
flight_JFK_delaynumber_df.rename(columns={'Time':'Date',                                   
                                          'temperature': 'Temperature (K)', 
                                          'humidity': 'Humidity(%)', 
                                          'pressure': 'Pressure(mb)',
                                          'wind_speed': 'Wind Speed(mph)',
                                          'wind_direction': 'Wind Direction'
                                         }, inplace=True)
flight_JFK_delaynumber_df.head()

,Date,delayed flights #,Temperature (K),Humidity(%),Pressure(mb),Wind Speed(mph),Wind Direction,Weather Description
0,2013-01-01,87,272.7,49.9,1019.9,6.1,272.2,broken clouds
1,2013-01-02,83,276.4,53.7,1012.2,6.4,268.8,overcast clouds
2,2013-01-03,97,270.7,58.0,1015.7,5.7,314.5,broken clouds
3,2013-01-04,92,271.1,55.0,1020.9,4.5,293.3,broken clouds
4,2013-01-05,85,274.0,52.3,1018.8,6.1,262.0,overcast clouds


In [7]:
# Save dataframe to sql
from sqlalchemy import create_engine

database_path = "NewYork_JFKflight_Delay2013_db"
engine = create_engine(f"sqlite:///{database_path}")
flight_JFK_delaynumber_df.to_sql("NewYork_JFKflight_Delay_2013", con=engine, if_exists='append')

## Conclusions:
1)	In 2013, the most delayed flights (217 in total) occurred under the weather condition ‘heavy snow’.

2)	High numbers of delayed flights not necessarily results from sever weather condition, for example, the second and third most delayed flights occurred under ‘light rain’ and ‘sky is clear’, therefore, other weather variables should also be considered, e.g. wind speed and direction.

In [8]:
flight_JFK_delaynumber_df.sort_values(['delayed flights #'],ascending=False).head()

,Date,delayed flights #,Temperature (K),Humidity(%),Pressure(mb),Wind Speed(mph),Wind Direction,Weather Description
66,2013-03-08,217,275.8,68.7,1016.2,8.0,139.5,heavy snow
220,2013-08-09,201,296.3,88.0,987.0,2.8,182.8,light rain
175,2013-06-25,200,300.3,75.0,1016.8,1.8,249.4,sky is clear
203,2013-07-23,200,298.3,85.3,1013.1,1.9,75.4,heavy intensity rain
163,2013-06-13,192,295.0,58.8,1008.5,2.6,266.6,sky is clear
